In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import skbio
import os

import asd
%matplotlib inline

In [ ]:
data_dir = "../microbiome-data/run1-4/vsearch-100/cd_even5721/"
uu_fp = os.path.join(data_dir, "bdiv_even5721/unweighted_unifrac_dm.txt")
wu_fp = os.path.join(data_dir, "bdiv_even5721/weighted_unifrac_dm.txt")
faithpd_fp = os.path.join(data_dir, "arare_max5721/alpha_div_collated/PD_whole_tree.txt")
sample_md_fp = 'combined-map.tsv'

In [ ]:
!md5 $uu_fp
!md5 $wu_fp
!md5 $faithpd_fp

In [ ]:
sample_md = pd.read_csv(sample_md_fp, sep='\t', index_col=0, dtype=object)
wu_dm = skbio.DistanceMatrix.read(wu_fp)
uu_dm = skbio.DistanceMatrix.read(uu_fp)

In [ ]:
faithpd = pd.read_csv(faithpd_fp, sep='\t')

In [ ]:
asd.get_donor_sids(sample_md)

In [ ]:
sample_md = asd.add_distance_to_donor(sample_md, uu_dm, metric_name='unweighted UniFrac')
sample_md = asd.add_distance_to_donor(sample_md, wu_dm, metric_name='weighted UniFrac')

In [ ]:
sample_md = asd.add_alpha_diversity(sample_md, faithpd, 'Faith PD')

In [ ]:
sample_md

## Change with time: swab

In [ ]:
asd.plot_week_data_with_stats(sample_md, sample_type='swab', metric='Faith PD', hue='responder')

## Change with time: stool

In [ ]:
asd.plot_week_data_with_stats(sample_md, sample_type='stool',
               metric='Faith PD', hue='responder')

In [ ]:
asd.plot_week_data_with_stats(sample_md, sample_type='stool',
               metric='Faith PD', hue='rectal_oral')

In [ ]:
asd.tabulate_week_to_week0_paired_stats(sample_md, sample_type='stool',
                                    metric='Faith PD')

In [ ]:
asd.tabulate_week_to_control_stats(sample_md, sample_type='stool',
                               metric='Faith PD')

## Engraftment: Swab

In [ ]:
asd.plot_week_data_with_stats(sample_md, sample_type='swab',
                          metric='unweighted UniFrac to initial donor', hue='responder', one_tailed=True,
                          hide_donor_baseline=True)

In [ ]:
asd.plot_week_data_with_stats(sample_md, sample_type='swab',
                          metric='unweighted UniFrac to most relevant donor', hue='responder', one_tailed=True,
                          hide_donor_baseline=True)

In [ ]:
asd.plot_week_data_with_stats(sample_md, sample_type='swab',
                          metric='unweighted UniFrac to most relevant donor', hue='rectal_oral', one_tailed=True,
                          hide_donor_baseline=True)

## Engraftment: stool

In [ ]:
asd.plot_week_data_with_stats(sample_md, sample_type='stool',
                          metric='unweighted UniFrac to initial donor', hue='responder', one_tailed=True,
                          hide_donor_baseline=True)

In [ ]:
asd.plot_week_data_with_stats(sample_md, sample_type='stool',
                          metric='unweighted UniFrac to most relevant donor', hue='responder', one_tailed=True,
                          hide_donor_baseline=True)

In [ ]:
asd.plot_week_data_with_stats(sample_md, sample_type='stool',
                          metric='unweighted UniFrac to most relevant donor', hue='rectal_oral', one_tailed=True,
                          hide_donor_baseline=True)

In [ ]:
asd.plot_week_data_with_stats(sample_md, sample_type='stool',
                          metric='weighted UniFrac to most relevant donor', hue='responder', one_tailed=False,
                          hide_donor_baseline=True)

In [ ]:
asd.plot_week_data_with_stats(sample_md, sample_type='swab',
                          metric='weighted UniFrac to most relevant donor', hue='responder', one_tailed=False,
                          hide_donor_baseline=True)